In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests

In [2]:
from bs4 import BeautifulSoup
from string import punctuation

In [3]:
response=requests.get('http://shakespeare.mit.edu/winters_tale/full.html')

In [4]:
text=BeautifulSoup(response.text)

In [5]:
text=text.get_text()

In [6]:
def clean_text(text):
    text=text.split('\n')
    text=[t.strip() for t in text if t.strip()!='']
    text=[''.join([cleanedtext for cleanedtext in t if cleanedtext not in punctuation]) for t in text if text]
    text=[t.strip() for t in text]
    text=[t for t in text if len(t.split())>7]
    return text

In [7]:
text=clean_text(text)

In [8]:
tok=tf.keras.preprocessing.text.Tokenizer()

In [9]:
tok.fit_on_texts(text)

In [10]:
seq=tok.texts_to_sequences(text)

In [11]:
seq=tf.keras.preprocessing.sequence.pad_sequences(seq,maxlen=5)

In [12]:
max(tok.word_index.values())

2866

In [13]:
seq

array([[ 111,    4,  572,  211,   45],
       [   8,  316,   35,   38,   45],
       [  15,    2,   12,  212,  126],
       ...,
       [ 216,  116,   40,   69, 2865],
       [  81,  465,    4,   25,  197],
       [ 723,    5,  109,  133,  159]], dtype=int32)

In [14]:
X,y=seq[:,:-1],seq[:,-1]

In [15]:
X.shape,y.shape

((1847, 4), (1847,))

In [20]:
model=tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=max(tok.word_index.values()),output_dim=5,input_length=4))
model.add(tf.keras.layers.LSTM(5))
model.add(tf.keras.layers.Dense(max(tok.word_index.values()),activation='softmax'))

In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4, 5)              14330     
_________________________________________________________________
lstm_1 (LSTM)                (None, 5)                 220       
_________________________________________________________________
dense_1 (Dense)              (None, 2866)              17196     
Total params: 31,746
Trainable params: 31,746
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(X,y)

Train on 1847 samples
